In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Coleta de dados

A base de dados tem 3 tabelas diferentes:
- pedidos (olist_orders_dataset)
- detalhes de pagamento dos pedidos (olist_order_payments_dataset)
- detalhes do consumidor que fez os pedidos (olist_customers_dataset)


In [ ]:
df_customer = pd.read_csv('dataset_casa/olist_customers_dataset.csv')
df_customer

In [ ]:
df_order = pd.read_csv('dataset_casa/olist_orders_dataset.csv')
df_order

In [ ]:
df_payment = pd.read_csv('dataset_casa/olist_order_payments_dataset.csv')
df_payment

## Objetivo da análise

- Quais os tipos de pagamentos utilizados pelos clientes?
- Qual o valor gasto em cada modalidade de pagamento?
- Qual estado mais pagou em boletos?
- Qual estado mais pagou em cartão de crédito?

In [ ]:
#Quais dados temos na tabela cliente:
df_customer.info()

In [ ]:
#Quais dados temos na tabela pedido:
df_order.info()

In [ ]:
#Quais dados temos na tabela pagamento:
df_payment.info()

In [ ]:
#Excluindo as colunas de id único do cliente e código postal que não será utilizada:
columns_drop_customer = ['customer_unique_id','customer_zip_code_prefix']
df_customer = df_customer.drop(columns_drop_customer, axis=1)
df_customer

In [ ]:
#Excluindo as colunas de aprovação de pedido, pedido entregue na transportadora, data de entrega do pedido, data estimada de entrega do pedido
columns_drop = ['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date' ,'order_estimated_delivery_date']
df_order = df_order.drop(columns_drop, axis=1)
df_order

In [ ]:
#Excluindo as colunas de pagamento subsequente, pagamento parcelado
columns_drop = ['payment_sequential', 'payment_installments']
df_payment = df_payment.drop(columns_drop, axis=1)
df_payment

In [ ]:
#Filtrando para apenas pedidos entregues
df_order = df_order.loc[df_order['order_status'] == 'delivered']
df_order

In [ ]:
#Resetando o index por conta da limpeza nos dados
df_order = df_order.reset_index(drop=True)
df_order

In [ ]:
#Transformando a coluna de data que esta em string para data e criando um nova coluna com mês
df_order['order_purchase_timestamp'] = pd.to_datetime(df_order['order_purchase_timestamp'])
df_order['month'] = df_order['order_purchase_timestamp'].dt.strftime('%Y-%m')
df_order

## Unindo o dataset
Para a análise proposta, irei utilizar as colunas de 'customer_id' e 'order_id'

In [ ]:
df_pay_order = pd.merge(df_payment, df_order, on='order_id')
df_pay_order

In [ ]:
df_final = pd.merge(df_pay_order, df_customer, on='customer_id')
df_final

In [ ]:
df_final.info()

In [ ]:
#Exportando a base final unificada
df_final.to_csv('base_unificada_casa.csv', index=False)

## Analisando os dados

### - Quais os tipos de pagamentos utilizados pelos clientes?
### - Qual o valor gasto em cada modalidade de pagamento?
### - Qual estado mais pagou em boletos?
### - Qual estado mais pagou em cartão de crédito?
### - Qual mês mais teve compras no cartão de débito?

In [ ]:
df_final['payment_type'].value_counts()

Podemos ver acima que temos 4 tipos de pagamentos utilizados pelos clientes: cartão de crédito, boleto, voucher e cartão de débito

In [ ]:
df_final.groupby(['payment_type']).agg({'payment_value': 'sum'}).sort_values('payment_value', ascending=False).T

In [ ]:
count_type = df_final['payment_type'].value_counts()
count_type.plot(kind='bar')
plt.title('Valor gasto por método de pagamento')
plt.xlabel('Método de pagamento')
plt.ylabel('Valor')

### Acima vemos não só os valores gastos em cada modalidade de pagamento como o maior valor pago foi no cartão de crédito

In [ ]:
plt.figure(figsize=(10, 5))

sns.countplot(data = df_final,
              x = 'customer_state',
              hue = 'payment_type')

plt.title('Tipos de pagamento por estado')
plt.xlabel('Estado')
plt.ylabel('Valor')
plt.legend(title = 'Método de pagamento')

plt.show()

In [ ]:
#SP foi o estado que mais fez compras
df_final.groupby(['customer_state']).agg({'payment_value': 'sum'}).sort_values('payment_value', ascending=False).head(1)

In [ ]:
#Qual mês mais teve compra?
df_compras = df_final.groupby('month').agg({'order_id': 'nunique'})
df_compras.sort_values(by='order_id', ascending=False).head(3)

In [ ]:
#Qual mês mais teve compras no cartão de débito?
df_debito = df_final.groupby(['payment_type', 'month']).agg({'month': 'size'}).reset_index().sort_values('size', ascending=False)
df_debito.loc[df_final['payment_type'] == 'debit_card']

In [ ]:
plt.figure(figsize=(12, 6))

sns.countplot(data = df_compras,
              x = 'month',
              hue = 'metodo_pagamento')

plt.title('Mês que mais teve compras no cartão de débito')
plt.xlabel('Mês')
plt.ylabel('Valor')

plt.xticks(rotation=330)



plt.show()